In [37]:
Packages <- c("dplyr",  "nleqslv", "broom","cubature", "geosphere", "data.table",  "ggplot2", "bbmle", "stringr",  "lubridate", "RColorBrewer")

invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0
source("scripts/ll_kt_both_bbmle.R")
source("scripts/ll_kt_both_grid_search.R")
source("scripts/LL_biophys.R")
source("scripts/GenGausKernInt_sum0.5.R") #integrate_kernel_sum1
source("scripts/GenGausKernInt_sum1.R")
source("scripts/cdf_solve.R") #median
source("scripts/cdf_solve90.R") #dist 90% retained

set.seed(7)
#read in the kernel fitting summary
kernels <- fread(file="script_output/KernelFits/summary_tables/kernel_fitting_summary.csv")
kernel2012_14 <- fread(file="empirical_data/genetics/GenKernelsForROMSComp2012-14.csv")

#read in the centroids adjusted for the simulation, so the Magbangons combined 
#centroids <- fread(file="script_output/SurveyData/SimulationCentroids.csv")
Centroids <- fread(file="empirical_data/site_centroids_SimTest.csv") 
Centroids$site <- gsub("_full", "", Centroids$site, fixed=TRUE)
Centroids$site <- gsub("_ten_per_cover", "", Centroids$site, fixed=TRUE)

Centroids <- Centroids %>%
    arrange(site)

#setorder(Centroids, site)#warning! This sets order based on site, and then lat/lon. So the table is not alphabetical by site, but that's fine as long as all of the "sampled_reef" vectors reflect this, so that reef_sizes, distance, and sampled_reefs match up by row/col index 
#read in the table with number of recruits sampled at each site for each year
AnnualRecsSamp <- fread(file="script_output/SurveyData/AnnualRecruitsSampled.csv")
#read in the table of the proportion of anemones sampled at each site for each year
PropSamp <- unique(fread(file="script_output/SurveyData/ProportionHabitatSampled.csv")[
    , .(site, year=end_year, prop_anem_samp=total_prop_hab_sampled_anems_tidied)][ #select and rename columns with the tideied data to use
    site %like% "Magbangon", site := "Magbangon"][ #collapse Magbangon values
    , prop_anem_samp := sum(prop_anem_samp), by=c("site", "year")], by=c("site", "year"))[ #collapse magbangons to match ROMS data
    site=="Sitio Lonas" & year %in% c(2012, 2013, 2014), prop_anem_samp :=1][site=="Caridad Proper" & year %in% c(2013, 2014), prop_anem_samp :=1]
                   
                   
### List of source locations
SitesSource <- Centroids

### List of destination locations
SitesDest <- Centroids

DistMatm <- distm(SitesSource[,c('lon','lat')], SitesSource[,c('lon','lat')], fun=distVincentyEllipsoid)
Distances <- DistMatm*10^-3
#read in the reef areas for the kernel fitting
Area <- fread("empirical_data/site_area_header_nonsurveyed_simulation_kernels_test.csv") %>%
    arrange(site) %>%
    filter(site %!in% c("near_north_full1", "near_north_full2", "near_north_full3", "near_south_full1", "near_south_full2", "near_south_full3")) %>%
    mutate(kmsq=msq*10^-6)# %>%
    #select(kmsq) #need to uncomment for functions to work
Area$site <- gsub("_ten_per_cover", "", Area$site, fixed=TRUE)

reef_sizes <- as.matrix(Area$kmsq)

#make a site index table, use this for Sampled_reefs input in kernel fitting
SiteIndex <- unique(Centroids %>% arrange(site), by="site")[, index := .I] #add the row number as the unique site index, leave CAI in if fitting a kernel 
SiteIndexBioPhys <- unique(Centroids %>% arrange(site), by="site")[site != "CAI" ][, index := .I] #add the row number as the unique site index, take CAI out for biophysical likelihood function

#make a table with the survey information for each site (how many fish sampled, prop anems sampled, total number of anems at site)
SurveyData <- AnnualRecsSamp[PropSamp, on=.(year, site)][#join the sampling tables together
    is.na(n_offs_gen), n_offs_gen := 0]#change NA's to 0
#setnames(SurveyData, c("PropAnemSamp", "TotalAnems"), c("prop_anem_samp", "total_anems"))
#setkey(SurveyData, site)
#check all sites are represented in centroids and area (and indirectly distances, which comes from centroids)
#Area[site %!in% centroids$site] #should be nothing

#Allison's abundance time series data 
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Script_outputs/females_df_F.RData?raw=true", destfile = "empirical_data/genetics/females_df_F.RData")
load("empirical_data/genetics/females_df_F.RData")
Abundance <- as.data.table(females_df_F)
setnames(Abundance, "nF", "num_females")
Abundance <- unique(Abundance[site %like% "Magbangon", site := "Magbangon"][ #collapse Magbangon values
            , num_females := sum(num_females), by=c("site", "year")], by=c("site", "year"))
#join the survey sampling tables together
SurveyData <- AnnualRecsSamp[PropSamp, on=.(year, site)][
    is.na(n_offs_gen), n_offs_gen := 0]#change NA's to 0


SurveyData <- Abundance[, c("year", "site", "num_females")][SurveyData, on=.(year, site)]#join in Allison's estimate of female abundance. There are NA values, but that's okay we can figure those out when we start thinking about incorporating uncertainty in this
#quick check that all components are in the same, alphabetical order
sum(which(SiteIndex$site==Area$site)==FALSE) #needs to be 0!! sites have to be in the same order
sum(which(Area$site==Centroids$site)==FALSE) #needs to be 0!! sites have to be in the same order

##read in genetic parentage matrices for each time frame MATCHING DIMENSIONS WITH BIOPHYSICAL FOR BIOPHYSICAL LIKELIHOOD- NOT KERNEL FITTING
#GenMat2012_4 <- as.matrix(fread(file="script_output/SurveyData/20210625_ParentageMatrix2012-14ForROMSComp.csv"))
#GenMat2012 <- as.matrix(fread(file="script_output/SurveyData/20210625_ParentageMatrix2012ForROMSComp.csv"))
#GenMat2013 <- as.matrix(fread(file="script_output/SurveyData/20210625_ParentageMatrix2013ForROMSComp.csv"))
#GenMat2014 <- as.matrix(fread(file="script_output/SurveyData/20210625_ParentageMatrix2014ForROMSComp.csv"))
#GenMatNEM <- as.matrix(fread(file="script_output/SurveyData/20210625_ParentageMatrixNEM2012-14ForROMSComp.csv"))
#GenMatSWM <- as.matrix(fread(file="script_output/SurveyData/20210625_ParentageMatrixSWM2012-14ForROMSComp.csv"))

#read in genetic parentage matrices for each time frame FOR KERNEL FITTING
KernelGenMat2012_4 <- as.matrix(fread(file="script_output/SurveyData/20210701_KernelParentageMatrix2012-14ForROMSComp.csv"))
KernelGenMat2012 <- as.matrix(fread(file="script_output/SurveyData/20210701_KernelParentageMatrix2012ForROMSComp.csv"))
KernelGenMat2013 <- as.matrix(fread(file="script_output/SurveyData/20210701_KernelParentageMatrix2013ForROMSComp.csv"))
KernelGenMat2014 <- as.matrix(fread(file="script_output/SurveyData/20210701_KernelParentageMatrix2014ForROMSComp.csv"))
KernelGenMatNEM <- as.matrix(fread(file="script_output/SurveyData/20210701_KernelParentageMatrixNEM2012-14ForROMSComp.csv"))
KernelGenMatSWM <- as.matrix(fread(file="script_output/SurveyData/20210701_KernelParentageMatrixSWM2012-14ForROMSComp.csv"))

###read in biophysical data for biophysical likelihood
#FullBiophysMatNorm <- as.matrix(fread(file="script_output/ROMSDataTables/2022-01-20_BioPhysNormConnMatrix2012-14ForROMSComp08DayPLD.csv"))
#AnnualBiophysMatNorm2012 <- as.matrix(fread(file="script_output/ROMSDataTables/2022-01-20_BioPhysNormConnMatrix2012ForROMSComp08DayPLD.csv"))
#AnnualBiophysMatNorm2013 <- as.matrix(fread(file="script_output/ROMSDataTables/2022-01-20_BioPhysNormConnMatrix2013ForROMSComp08DayPLD.csv"))
#AnnualBiophysMatNorm2014 <- as.matrix(fread(file="script_output/ROMSDataTables/2022-01-20_BioPhysNormConnMatrix2014ForROMSComp08DayPLD.csv"))
#MonsoonBiophysMatNormNEM <- as.matrix(fread(file="script_output/ROMSDataTables/2022-01-20_BioPhysNormConnMatrixNEMForROMSComp08DayPLD.csv" ))
#MonsoonBiophysMatNormSWM <-as.matrix(fread(file="script_output/ROMSDataTables/2022-01-20_BioPhysNormConnMatrixSWMForROMSComp08DayPLD.csv" ))

##read in biophysical data for KERNEL FITTING
FullBiophysMat <- as.matrix(fread(file="script_output/ROMSDataTables/2022-01-20_BioPhysParentageMatrix2012-14ForROMSComp08DayPLD.csv"))
#fwrite(biophys_parentage_matrices$file="script_output/ROMSDataTables/2022-01-20_BioPhysParentageMatrix2012-14ForROMSComp08DayPLD.csv")

AnnualBiophysMat2012 <- as.matrix(fread(file="script_output/ROMSDataTables/2022-01-20_BioPhysParentageMatrix2012ForROMSComp08DayPLD.csv"))
AnnualBiophysMat2013 <- as.matrix(fread(file="script_output/ROMSDataTables/2022-01-20_BioPhysParentageMatrix2013ForROMSComp08DayPLD.csv"))
AnnualBiophysMat2014 <- as.matrix(fread(file="script_output/ROMSDataTables/2022-01-20_BioPhysParentageMatrix2014ForROMSComp08DayPLD.csv"))
MonsoonBiophysMatNEM <- as.matrix(fread(file="script_output/ROMSDataTables/2022-01-20_BioPhysParentageMatrixNEMForROMSComp08DayPLD.csv"))
MonsoonBiophysMatSWM <- as.matrix(fread(file="script_output/ROMSDataTables/2022-01-20_BioPhysParentageMatrixSWMForROMSComp08DayPLD.csv"))

##15 DAY PLD SIMULATIONS
##read in biophysical data for biophysical likelihood
#FullBiophysMatNorm <- as.matrix(fread(file="script_output/ROMSDataTables/20210917_BioPhysNormConnMatrix2012-14ForROMSComp15DayPLD.csv"))
#AnnualBiophysMatNorm2012 <- as.matrix(fread(file="script_output/ROMSDataTables/20210917_BioPhysNormConnMatrix2012ForROMSComp15DayPLD.csv"))
#AnnualBiophysMatNorm2013 <- as.matrix(fread(file="script_output/ROMSDataTables/20210917_BioPhysNormConnMatrix2013ForROMSComp15DayPLD.csv"))
#AnnualBiophysMatNorm2014 <- as.matrix(fread(file="script_output/ROMSDataTables/20210917_BioPhysNormConnMatrix2014ForROMSComp15DayPLD.csv"))
#MonsoonBiophysMatNormNEM <- as.matrix(fread(file="script_output/ROMSDataTables/20210917_BioPhysNormConnMatrixNEM2012-14ForROMSComp15DayPLD.csv"))
#MonsoonBiophysMatNormSWM <-as.matrix(fread(file="script_output/ROMSDataTables/20210917_BioPhysNormConnMatrixSWM2012-14ForROMSComp15DayPLD.csv"))

##read in biophysical data for KERNEL FITTING
#FullBiophysMat <- as.matrix(fread(file="script_output/ROMSDataTables/20210917_BioPhysParentageMatrix2012-14ForROMSComp15DayPLD.csv"))
#AnnualBiophysMat2012 <- as.matrix(fread(file="script_output/ROMSDataTables/20210917_BioPhysParentageMatrix2012ForROMSComp15DayPLD.csv"))
#AnnualBiophysMat2013 <- as.matrix(fread(file="script_output/ROMSDataTables/20210917_BioPhysParentageMatrix2013ForROMSComp15DayPLD.csv"))
#AnnualBiophysMat2014 <- as.matrix(fread(file="script_output/ROMSDataTables/20210917_BioPhysParentageMatrix2014ForROMSComp15DayPLD.csv"))
#MonsoonBiophysMatNEM <- as.matrix(fread(file="script_output/ROMSDataTables/20210917_BioPhysParentageMatrixNEM2012-14ForROMSComp15DayPLD.csv"))
#MonsoonBiophysMatSWM <- as.matrix(fread(file="script_output/ROMSDataTables/20210917_BioPhysParentageMatrixSWM2012-14ForROMSComp15DayPLD.csv"))

[1] 0

[1] 0

In [20]:
sum(AnnualBiophysMat2012)+sum(AnnualBiophysMat2013)+sum(AnnualBiophysMat2014)==sum(FullBiophysMat)#should be true

[1] TRUE

In [49]:
sum(MonsoonBiophysMatNEM)
sum(MonsoonBiophysMatSWM)

[1] 827349

[1] 127914

In [45]:


#fit the kernels, get the biophysical data together
biophys_par_data2012 <- list(Distances=Distances, Assignments=AnnualBiophysMat2012, Sampled_reefs=t(as.matrix(SiteIndex[site %in% colnames(AnnualBiophysMat2012), .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(AnnualBiophysMat2012), ncol=1, 1))
Sim2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2012, control=list(maxit=500)))

biophys_par_data2013 <- list(Distances=Distances, Assignments=AnnualBiophysMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% colnames(AnnualBiophysMat2013), .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(AnnualBiophysMat2013), ncol=1, 1))
Sim2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2013, control=list(maxit=500)))

biophys_par_data2014 <- list(Distances=Distances, Assignments=AnnualBiophysMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% colnames(AnnualBiophysMat2014), .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(AnnualBiophysMat2014), ncol=1, 1))
Sim2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2014, control=list(maxit=500)))

biophys_par_data2012_4 <- list(Distances=Distances, Assignments=FullBiophysMat, Sampled_reefs=t(as.matrix(SiteIndex[site %in% colnames(FullBiophysMat), .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(FullBiophysMat), ncol=1, 1))
Sim2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2012_4, control=list(maxit=500)))

biophys_par_dataNEM <- list(Distances=Distances, Assignments=MonsoonBiophysMatNEM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% colnames(FullBiophysMat), .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(MonsoonBiophysMatNEM), ncol=1, 1))
SimNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-.3, theta=.6), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_dataNEM, control=list(maxit=500)))

biophys_par_dataSWM <- list(Distances=Distances, Assignments=MonsoonBiophysMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% colnames(FullBiophysMat), .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(MonsoonBiophysMatSWM), ncol=1, 1))
SimSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-.3, theta=.6), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_dataSWM, control=list(maxit=500)))



In [53]:
t(as.matrix(SiteIndex[site %in% colnames(FullBiophysMat)]))
t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site]]))


site,Cabatoan,Caridad Cemetery,Caridad Proper,Elementary School,Gabas,Haina,Hicgop South,Magbangon,Palanas,Poroc Rose,Poroc San Flower,San Agustin,Sitio Baybayon,Sitio Lonas,Sitio Tugas,Tamakin Dacot,Visca,Wangag
lon,124.7270,124.7469,124.7508,124.7684,124.7934,124.7813,124.7635,124.7253,124.7113,124.7852,124.7853,124.7836,124.7755,124.7830,124.7664,124.7838,124.7865,124.7152
lat,10.8554,10.8363,10.8306,10.7989,10.7279,10.6409,10.8137,10.8569,10.8737,10.7555,10.7641,10.7690,10.6365,10.7744,10.8060,10.6526,10.7442,10.8707
index,1,3,4,9,10,11,12,13,20,21,22,23,24,25,26,27,28,29


site,Cabatoan,Caridad Cemetery,Caridad Proper,Elementary School,Haina,Hicgop South,Magbangon,Palanas,Poroc Rose,Poroc San Flower,San Agustin,Sitio Baybayon,Sitio Lonas,Tamakin Dacot,Visca,Wangag
lon,124.7270,124.7469,124.7508,124.7684,124.7813,124.7635,124.7253,124.7113,124.7852,124.7853,124.7836,124.7755,124.7830,124.7838,124.7865,124.7152
lat,10.8554,10.8363,10.8306,10.7989,10.6409,10.8137,10.8569,10.8737,10.7555,10.7641,10.7690,10.6365,10.7744,10.6526,10.7442,10.8707
index,1,3,4,9,11,12,13,20,21,22,23,24,25,27,28,29


In [ ]:
biophys_par_dataSWM <- list(Distances=Distances, Assignments=biophys_parentage_matrices$MonsoonBiophysMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$MonsoonBiophysMatSWM), ncol=1, 1))
biophys_par_dataSWM <- list(Distances=Distances, Assignments=MonsoonBiophysMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% colnames(FullBiophysMat), .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(MonsoonBiophysMatSWM), ncol=1, 1))


In [46]:
SimNEMFit
SimSWMFit


Call:
mle2(minuslogl = LL_kt_bbmle, start = list(k = -0.3, theta = 0.6), 
    method = "L-BFGS-B", data = biophys_par_dataNEM, lower = c(-10, 
        0.15), upper = c(10, 5), control = list(maxit = 500))

Coefficients:
         k      theta 
-0.4314953  0.6569211 

Log-likelihood: -1864791 


Call:
mle2(minuslogl = LL_kt_bbmle, start = list(k = -0.3, theta = 0.6), 
    method = "L-BFGS-B", data = biophys_par_dataSWM, lower = c(-10, 
        0.15), upper = c(10, 5), control = list(maxit = 500))

Coefficients:
        k     theta 
0.2398232 0.4672805 

Log-likelihood: -128013.1 

In [47]:
BestKNEM <- as.numeric(coef(SimNEMFit)[1])
BestThetaNEM <- as.numeric(coef(SimNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
MDDNEM

[1] 3.617709

In [48]:
BestKSWM <- as.numeric(coef(SimSWMFit)[1])
BestThetaSWM <- as.numeric(coef(SimSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
MDDSWM

[1] 6.355059

In [36]:

#make a summary table for all time frames of simulation data
SimKernels <- as.data.frame(matrix(NA, nrow = 6, ncol = 8))
setDT(SimKernels)
setnames(SimKernels, c("time_scale", "time_id", "k", "theta", "mdd", "med", "dist90", "log_like"))
#SimKernels <- data.table(time_scale=character(), time_id=character(), k=numeric(), theta=numeric(), mdd=numeric(), med=numeric(), dist90=numeric())


BestK2012 <- as.numeric(coef(Sim2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Sim2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- nleqslv(x = 7, fn = cdf_solve)$x 
Dist90_2012 <- nleqslv(x = 7, fn = cdf_solve90)$x

SimKernels$time_scale[1] <- "annual"
SimKernels$time_id[1] <- "2012"
SimKernels$k[1] <-  BestK2012
SimKernels$theta[1] <- BestTheta2012
SimKernels$mdd[1] <- MDD2012
SimKernels$med[1] <- Med2012
SimKernels$dist90[1] <- Dist90_2012
SimKernels$log_like[1] <- logLik(Sim2012Fit)[1]

BestK2013 <- as.numeric(coef(Sim2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Sim2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- nleqslv(x = 7, fn = cdf_solve)$x 
Dist90_2013 <- nleqslv(x = 7, fn = cdf_solve90)$x

SimKernels$time_scale[2] <- "annual"
SimKernels$time_id[2] <- "2013"
SimKernels$k[2] <-  BestK2013
SimKernels$theta[2] <- BestTheta2013
SimKernels$mdd[2] <- MDD2013
SimKernels$med[2] <- Med2013
SimKernels$dist90[2] <- Dist90_2013
SimKernels$log_like[2] <- logLik(Sim2013Fit)[1]


BestK2014 <- as.numeric(coef(Sim2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Sim2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- nleqslv(x = 7, fn = cdf_solve)$x
Dist90_2014 <- nleqslv(x = 7, fn = cdf_solve90)$x

SimKernels$time_scale[3] <- "annual"
SimKernels$time_id[3] <- "2014"
SimKernels$k[3] <-  BestK2014
SimKernels$theta[3] <- BestTheta2014
SimKernels$mdd[3] <- MDD2014
SimKernels$med[3] <- Med2014
SimKernels$dist90[3] <- Dist90_2014
SimKernels$log_like[3] <- logLik(Sim2014Fit)[1]

BestK2012_4 <- as.numeric(coef(Sim2012_4Fit)[1])
BestTheta2012_4 <- as.numeric(coef(Sim2012_4Fit)[2])
MDD2012_4 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012_4, theta=BestTheta2012_4, , method = "pcubature")$integral)
k_eval <- BestK2012_4
theta_eval <- BestTheta2012_4
Med2012_4  <- nleqslv(x = 7, fn = cdf_solve)$x 
Dist90_2012_4 <- nleqslv(x = 7, fn = cdf_solve90)$x

SimKernels$time_scale[4] <- "interannual"
SimKernels$time_id[4] <- "2012-4"
SimKernels$k[4] <-  BestK2012_4
SimKernels$theta[4] <- BestTheta2012_4
SimKernels$mdd[4] <- MDD2012_4
SimKernels$med[4] <- Med2012_4
SimKernels$dist90[4] <- Dist90_2012_4
SimKernels$log_like[4] <- logLik(Sim2012_4Fit)[1]

BestKNEM <- as.numeric(coef(SimNEMFit)[1])
BestThetaNEM <- as.numeric(coef(SimNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- nleqslv(x = 7, fn = cdf_solve)$x 
Dist90_NEM <- nleqslv(x = 7, fn = cdf_solve90)$x

SimKernels$time_scale[5] <- "season"
SimKernels$time_id[5] <- "NEM"
SimKernels$k[5] <-  BestKNEM
SimKernels$theta[5] <- BestThetaNEM
SimKernels$mdd[5] <- MDDNEM
SimKernels$med[5] <- MedNEM
SimKernels$dist90[5] <- Dist90_NEM
SimKernels$log_like[5] <- logLik(SimNEMFit)[1]

BestKSWM <- as.numeric(coef(SimSWMFit)[1])
BestThetaSWM <- as.numeric(coef(SimSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- nleqslv(x = 7, fn = cdf_solve)$x 
Dist90_SWM <- nleqslv(x = 7, fn = cdf_solve90)$x

SimKernels$time_scale[6] <- "season"
SimKernels$time_id[6] <- "SWM"
SimKernels$k[6] <-  BestKSWM
SimKernels$theta[6] <- BestThetaSWM
SimKernels$mdd[6] <- MDDSWM
SimKernels$med[6] <- MedSWM
SimKernels$dist90[6] <- Dist90_SWM
SimKernels$log_like[6] <- logLik(SimSWMFit)[1]

SimKernels[, `:=`(k=k, theta=theta, mdd=mdd, med=med, dist90=dist90, data_source="simulation"), by="time_id"]

#fwrite(SimKernels, file="script_output/KernelFits/SimulationKernelFits08DayPLD.csv")


SimKernels


time_scale,time_id,k,theta,mdd,med,dist90,log_like,data_source
annual,2012,-0.38751254,0.6370261,3.778224,2.096410,9.305059,-725758.2,simulation
annual,2013,-0.64237719,0.6792543,4.080705,2.351905,9.967812,-662951.4,simulation
annual,2014,0.07565334,0.5641438,3.504783,1.797263,8.749074,-629397.4,simulation
interannual,2012-4,-0.31231809,0.6214930,3.770613,2.060501,9.314044,-2021633.6,simulation
season,NEM,-0.43151979,0.6569262,3.617719,2.044842,8.875493,-1864791.3,simulation
season,SWM,0.23883526,0.4673893,6.354512,2.826102,16.085780,-128013.1,simulation


In [14]:
head(SurveyData)

year,site,num_females,n_offs_gen,prop_anem_samp
2012,Cabatoan,8,4,0.4230769
2012,Caridad Cemetery,NA,0,0.0000000
2012,Caridad Proper,NA,0,0.0000000
2012,Elementary School,NA,0,0.0000000
2012,Gabas,NA,0,0.0000000
2012,Haina,NA,0,0.0000000


In [18]:
#write out supplemental tables for paper
#fwrite(SurveyData[year %in% c("2012", "2013", "2014") & site != "Gabas", .(year, site, n_offs_gen, prop_anem_samp=round(prop_anem_samp, 3))], "script_output/SurveyData/SamplingTableSupplment.csv")

#fwrite(SurveyData[year %in% c("2012", "2013", "2014") & site != "Gabas",.(avg_num_females=round(mean(num_females, na.rm = TRUE),0)), by=site], "script_output/SurveyData/AvgNTableSupplment.csv")

In [7]:
, 1)

year,site,num_females,n_offs_gen,prop_anem_samp
2012,Gabas,NA,0,0.0000000
2013,Gabas,NA,0,0.0000000
2014,Gabas,NA,0,0.0000000
2015,Gabas,8,1,0.1111111
2016,Gabas,12,1,0.1777778
2017,Gabas,21,5,0.2592593
2018,Gabas,NA,0,0.2222222


In [5]:
#genetic data with same reef metadata structure as model

par_data2012 <- list(Distances=Distances, Assignments=KernelGenMat2012, Sampled_reefs=t(as.matrix(SiteIndex[site %in% colnames(KernelGenMat2012), .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2012 & site %in% colnames(KernelGenMat2012), .(prop_anem_samp)]))
Gen2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2012, control=list(maxit=500)))

par_data2013 <- list(Distances=Distances, Assignments=KernelGenMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% colnames(KernelGenMat2013), .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2013 & site %in% colnames(KernelGenMat2013), .(prop_anem_samp)]))
Gen2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2013, control=list(maxit=500)))

par_data2014 <- list(Distances=Distances, Assignments=KernelGenMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% colnames(KernelGenMat2014), .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & site %in% colnames(KernelGenMat2014), .(prop_anem_samp)]))
Gen2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2014, control=list(maxit=500)))

par_data2012_4 <- list(Distances=Distances, Assignments=KernelGenMat2012_4, Sampled_reefs=t(as.matrix(SiteIndex[site %in% colnames(KernelGenMat2012_4), .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & site %in% colnames(KernelGenMat2012_4), .(prop_anem_samp)]))
Gen2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2012_4, control=list(maxit=500)))

par_dataNEM <- list(Distances=Distances, Assignments=KernelGenMatNEM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% colnames(KernelGenMatNEM), .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & site %in% colnames(KernelGenMat2012_4), .(prop_anem_samp)]))
GenNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_dataNEM, control=list(maxit=500)))

par_dataSWM <- list(Distances=Distances, Assignments=KernelGenMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% colnames(KernelGenMatNEM), .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & site %in% colnames(KernelGenMat2012_4), .(prop_anem_samp)]))
GenSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_dataSWM, control=list(maxit=500)))



In [6]:
#make a summary table for all time frames of genetic data
GenKernels <- as.data.frame(matrix(NA, nrow = 6, ncol = 8))
setDT(GenKernels)
setnames(GenKernels, c("time_scale", "time_id", "k", "theta", "mdd", "med", "dist90", "log_like"))
#GenKernels <- data.table(time_scale=character(), time_id=character(), k=numeric(), theta=numeric(), mdd=numeric(), med=numeric(), dist90=numeric())


BestK2012 <- as.numeric(coef(Gen2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Gen2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

GenKernels$time_scale[1] <- "annual"
GenKernels$time_id[1] <- "2012"
GenKernels$k[1] <-  BestK2012
GenKernels$theta[1] <- BestTheta2012
GenKernels$mdd[1] <- MDD2012
GenKernels$med[1] <- Med2012
GenKernels$dist90[1] <- Dist90_2012
GenKernels$log_like[1] <- logLik(Gen2012Fit)[1]

BestK2013 <- as.numeric(coef(Gen2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Gen2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

GenKernels$time_scale[2] <- "annual"
GenKernels$time_id[2] <- "2013"
GenKernels$k[2] <-  BestK2013
GenKernels$theta[2] <- BestTheta2013
GenKernels$mdd[2] <- MDD2013
GenKernels$med[2] <- Med2013
GenKernels$dist90[2] <- Dist90_2013
GenKernels$log_like[2] <- logLik(Gen2013Fit)[1]

BestK2014 <- as.numeric(coef(Gen2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Gen2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

GenKernels$time_scale[3] <- "annual"
GenKernels$time_id[3] <- "2014"
GenKernels$k[3] <-  BestK2014
GenKernels$theta[3] <- BestTheta2014
GenKernels$mdd[3] <- MDD2014
GenKernels$med[3] <- Med2014
GenKernels$dist90[3] <- Dist90_2014
GenKernels$log_like[3] <- logLik(Gen2014Fit)[1]

BestK2012_4 <- as.numeric(coef(Gen2012_4Fit)[1])
BestTheta2012_4 <- as.numeric(coef(Gen2012_4Fit)[2])
MDD2012_4 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012_4, theta=BestTheta2012_4, , method = "pcubature")$integral)
k_eval <- BestK2012_4
theta_eval <- BestTheta2012_4
Med2012_4  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012_4 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

GenKernels$time_scale[4] <- "interannual"
GenKernels$time_id[4] <- "2012-4"
GenKernels$k[4] <-  BestK2012_4
GenKernels$theta[4] <- BestTheta2012_4
GenKernels$mdd[4] <- MDD2012_4
GenKernels$med[4] <- Med2012_4
GenKernels$dist90[4] <- Dist90_2012_4
GenKernels$log_like[4] <- logLik(Gen2012_4Fit)[1]

BestKNEM <- as.numeric(coef(GenNEMFit)[1])
BestThetaNEM <- as.numeric(coef(GenNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

GenKernels$time_scale[5] <- "season"
GenKernels$time_id[5] <- "NEM"
GenKernels$k[5] <-  BestKNEM
GenKernels$theta[5] <- BestThetaNEM
GenKernels$mdd[5] <- MDDNEM
GenKernels$med[5] <- MedNEM
GenKernels$dist90[5] <- Dist90_NEM
GenKernels$log_like[5] <- logLik(GenNEMFit)[1]

BestKSWM <- as.numeric(coef(GenSWMFit)[1])
BestThetaSWM <- as.numeric(coef(GenSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

GenKernels$time_scale[6] <- "season"
GenKernels$time_id[6] <- "SWM"
GenKernels$k[6] <-  BestKSWM
GenKernels$theta[6] <- BestThetaSWM
GenKernels$mdd[6] <- MDDSWM
GenKernels$med[6] <- MedSWM
GenKernels$dist90[6] <- Dist90_SWM
GenKernels$log_like[6] <- logLik(GenSWMFit)[1]

GenKernels[, `:=`(k=round(k, 5), theta=round(theta, 5), mdd=round(mdd, 2), med=round(med, 2), dist90=round(dist90, 2), data_source="genetics"), by="time_id"]

#fwrite(GenKernels, file="script_output/KernelFits/summary_tables/GeneticKernelFits.csv")


GenKernels



time_scale,time_id,k,theta,mdd,med,dist90,log_like,data_source
annual,2012,-3.27091,3.52876,13.02,12.02,25.08,-17.26499,genetics
annual,2013,-3.27484,5.00000,12.77,12.18,23.91,-112.75593,genetics
annual,2014,1.97231,0.26016,76.00,17.47,182.37,-78.48620,genetics
interannual,2012-4,-0.22643,0.43900,13.69,5.77,34.74,-218.61429,genetics
season,NEM,0.76361,0.37294,12.87,4.64,32.61,-27.26112,genetics
season,SWM,-3.38408,5.00000,14.25,13.59,26.67,-71.81265,genetics


__next three cells are copied and pasted from my github code for this notebook when it worked in september. if it doesn't work as expected again, troubleshoot from there__

In [ ]:
options(digits=15)
# use a grid search to find k that minimizes the log likelihood
#LogLikeSim2012 <- setDT(expand.grid(k_eval=seq(from=-5, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 200)))
#LogLikeSim2012[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_data2012$Distances, Assignments=biophys_par_data2012$Assignments, Sampled_reefs=biophys_par_data2012$Sampled_reefs, Reef_sizes=biophys_par_data2012$Reef_sizes, Adult_sample_proportions=biophys_par_data2012$Adult_sample_proportions), by=.(k_eval, theta_eval)]   #write profile results
fwrite(LogLikeSim2012, file="script_output/KernelFits/profiles/LikelihoodProfile-Biophysical-2012-08DayPLD.csv")

LogLikeSim2013 <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 20000)))
LogLikeSim2013[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_data2013$Distances, Assignments=biophys_par_data2013$Assignments, Sampled_reefs=biophys_par_data2013$Sampled_reefs, Reef_sizes=biophys_par_data2013$Reef_sizes, Adult_sample_proportions=biophys_par_data2013$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
#write profile results
fwrite(LogLikeSim2013, file="script_output/KernelFits/profiles/LikelihoodProfile-Biophysical-2013-08DayPLD.csv")

LogLikeSim2014 <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 20000)))
LogLikeSim2014[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_data2014$Distances, Assignments=biophys_par_data2014$Assignments, Sampled_reefs=biophys_par_data2014$Sampled_reefs, Reef_sizes=biophys_par_data2014$Reef_sizes, Adult_sample_proportions=biophys_par_data2014$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
#write profile results
fwrite(LogLikeSim2014, file="script_output/KernelFits/profiles/LikelihoodProfile-Biophysical-2014-08DayPLD.csv")

LogLikeSim2012_4 <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 20000)))
LogLikeSim2012_4[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_data2012_4$Distances, Assignments=biophys_par_data2012_4$Assignments, Sampled_reefs=biophys_par_data2012_4$Sampled_reefs, Reef_sizes=biophys_par_data2012_4$Reef_sizes, Adult_sample_proportions=biophys_par_data2012_4$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
#write profile results
fwrite(LogLikeSim2012_4, file="script_output/KernelFits/profiles/LikelihoodProfile-Biophysical-2012-4-08DayPLD.csv")

LogLikeSimNEM <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 20000)))
LogLikeSimNEM[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_dataNEM$Distances, Assignments=biophys_par_dataNEM$Assignments, Sampled_reefs=biophys_par_dataNEM$Sampled_reefs, Reef_sizes=biophys_par_dataNEM$Reef_sizes, Adult_sample_proportions=biophys_par_dataNEM$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
#write profile results
fwrite(LogLikeSimNEM, file="script_output/KernelFits/profiles/LikelihoodProfile-Biophysical-NEM-08DayPLD.csv")

LogLikeSimSWM <- setDT(expand.grid(k_eval=seq(from=-5, to=5, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 20000)))
LogLikeSimSWM[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_dataSWM$Distances, Assignments=biophys_par_dataSWM$Assignments, Sampled_reefs=biophys_par_dataSWM$Sampled_reefs, Reef_sizes=biophys_par_dataSWM$Reef_sizes, Adult_sample_proportions=biophys_par_dataSWM$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
#write profile results
fwrite(LogLikeSimSWM, file="script_output/KernelFits/profiles/LikelihoodProfile-Biophysical-SWM-08DayPLD.csv")

##also for genetics
#LogLikeGen2012 <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 200)))
#LogLikeGen2012[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=par_data2012$Distances, Assignments=par_data2012$Assignments, Sampled_reefs=par_data2012$Sampled_reefs, Reef_sizes=par_data2012$Reef_sizes, Adult_sample_proportions=par_data2012$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
##write profile results
#fwrite(LogLikeGen2012, file="script_output/KernelFits/profiles/LikelihoodProfileGenetic2012.csv")
#
#LogLikeGen2013 <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 200)))
#LogLikeGen2013[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=par_data2013$Distances, Assignments=par_data2013$Assignments, Sampled_reefs=par_data2013$Sampled_reefs, Reef_sizes=par_data2013$Reef_sizes, Adult_sample_proportions=par_data2013$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
##write profile results
#fwrite(LogLikeGen2013, file="script_output/KernelFits/profiles/LikelihoodProfileGenetic2013.csv")
#
#LogLikeGen2014 <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 200)))
#LogLikeGen2014[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=par_data2014$Distances, Assignments=par_data2014$Assignments, Sampled_reefs=par_data2014$Sampled_reefs, Reef_sizes=par_data2014$Reef_sizes, Adult_sample_proportions=par_data2014$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
##write profile results
#fwrite(LogLikeGen2014, file="script_output/KernelFits/profiles/LikelihoodProfileGenetic2014.csv")
#
#LogLikeGen2012_4 <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 200)))
#LogLikeGen2012_4[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=par_data2012_4$Distances, Assignments=par_data2012_4$Assignments, Sampled_reefs=par_data2012_4$Sampled_reefs, Reef_sizes=par_data2012_4$Reef_sizes, Adult_sample_proportions=par_data2012_4$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
##write profile results
#fwrite(LogLikeGen2012_4, file="script_output/KernelFits/profiles/LikelihoodProfileGenetic2012_4.csv")
#
#LogLikeGenNEM <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 200)))
#LogLikeGenNEM[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=par_dataNEM$Distances, Assignments=par_dataNEM$Assignments, Sampled_reefs=par_dataNEM$Sampled_reefs, Reef_sizes=par_dataNEM$Reef_sizes, Adult_sample_proportions=par_dataNEM$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
##write profile results
#fwrite(LogLikeGenNEM, file="script_output/KernelFits/profiles/LikelihoodProfileGeneticNEM.csv")
#
#LogLikeGenSWM <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 200)))
#LogLikeGenSWM[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=par_dataSWM$Distances, Assignments=par_dataSWM$Assignments, Sampled_reefs=par_dataSWM$Sampled_reefs, Reef_sizes=par_dataSWM$Reef_sizes, Adult_sample_proportions=par_dataSWM$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
##write profile results
#fwrite(LogLikeGenSWM, file="script_output/KernelFits/profiles/LikelihoodProfileGeneticSWM.csv")


In [ ]:
LogLikeSim2012 <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 20000)))
LogLikeSim2012[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_data2012$Distances, Assignments=biophys_par_data2012$Assignments, Sampled_reefs=biophys_par_data2012$Sampled_reefs, Reef_sizes=biophys_par_data2012$Reef_sizes, Adult_sample_proportions=biophys_par_data2012$Adult_sample_proportions), by=.(k_eval, theta_eval)]   #write profile results


In [61]:
head(LogLikeSim2012)
tail(LogLikeSim2012[log_like < cutoff_Sim2012])

k_eval,theta_eval,log_like
-10.00000000000000,0.1,1401457.68451435
-9.98999499749875,0.1,1401413.45217715
-9.97998999499750,0.1,1401369.17564605
-9.96998499249625,0.1,1401324.85487698
-9.95997998999500,0.1,1401280.48982588
-9.94997498749375,0.1,1401236.08044861


k_eval,theta_eval,log_like
-0.415207603801901,0.642702135106755,725766.277920341
-0.415207603801901,0.642947147357368,725768.345962222
-0.415207603801901,0.643192159607980,725771.644617087
-0.425212606303152,0.644172208610431,725772.081192132
-0.425212606303152,0.644417220861043,725771.650369160
-0.425212606303152,0.644662233111656,725772.437680529


In [31]:
LL_kt_grid(k=-10, theta=.1, Distances=biophys_par_data2012$Distances, Assignments=biophys_par_data2012$Assignments, Sampled_reefs=biophys_par_data2012$Sampled_reefs, Reef_sizes=biophys_par_data2012$Reef_sizes, Adult_sample_proportions=biophys_par_data2012$Adult_sample_proportions)  #write profile results

[1] 1401458

In [ ]:
##genetics
#cutoff_Gen2012 <- min(LogLikeGen2012$log_like) + qchisq(.90, 2)/2 #use the 95% quantile of the chi-sq distribution with two df because it's bivariate
#
#cutoff_Gen2013 <- min(LogLikeGen2013$log_like) + qchisq(.90, 2)/2 #use the 95% quantile of the chi-sq distribution with two df because it's bivariate
#
#cutoff_Gen2014 <- min(LogLikeGen2014$log_like) + qchisq(.90, 2)/2 #use the 95% quantile of the chi-sq distribution with two df because it's bivariate
#
#cutoff_Gen2012_4 <- min(LogLikeGen2012_4$log_like) + qchisq(.90, 2)/2 #use the 95% quantile of the chi-sq distribution with two df because it's bivariate
#
#cutoff_GenNEM <- min(LogLikeGenNEM$log_like) + qchisq(.90, 2)/2 #use the 95% quantile of the chi-sq distribution with two df because it's bivariate
#
#cutoff_GenSWM <- min(LogLikeGenSWM$log_like) + qchisq(.90, 2)/2 #use the 95% quantile of the chi-sq distribution with two df because it's bivariate
#
#biophysical simulation
cutoff_Sim2012 <- min(LogLikeSim2012$log_like) + qchisq(.90, 2)/2 #use the 95% quantile of the chi-sq distribution with two df because it's bivariate

cutoff_Sim2013 <- min(LogLikeSim2013$log_like) + qchisq(.90, 2)/2 #use the 95% quantile of the chi-sq distribution with two df because it's bivariate

cutoff_Sim2014 <- min(LogLikeSim2014$log_like) + qchisq(.90, 2)/2 #use the 95% quantile of the chi-sq distribution with two df because it's bivariate

cutoff_Sim2012_4 <- min(LogLikeSim2012_4$log_like) + qchisq(.90, 2)/2 #use the 95% quantile of the chi-sq distribution with two df because it's bivariate

cutoff_SimNEM <- min(LogLikeSimNEM$log_like) + qchisq(.90, 2)/2 #use the 95% quantile of the chi-sq distribution with two df because it's bivariate

cutoff_SimSWM <- min(LogLikeSimSWM$log_like) + qchisq(.90, 2)/2 #use the 95% quantile of the chi-sq distribution with two df because it's bivariate


In [55]:
SimKernels

time_scale,time_id,k,theta,mdd,med,dist90,log_like,data_source
annual,2012,-0.3875125382529303,0.637026052468532,3.77822423644382,2.09641041113457,9.30505888780661,-725758.169795862,simulation
annual,2013,-0.6423771894499601,0.679254254139089,4.08070541961774,2.35190528319043,9.96781212981102,-662951.384904839,simulation
annual,2014,0.0756533363405335,0.564143762474149,3.50478307903728,1.79726295628089,8.74907418644373,-629397.429098801,simulation
interannual,2012-4,-0.3123180929957831,0.621492970172060,3.77061337467421,2.06050058551574,9.31404394988521,-2021633.646682530,simulation
season,NEM,-0.4315197928815913,0.656926228161586,3.61771857692861,2.04484175562624,8.87549325337996,-1864791.296696837,simulation
season,SWM,0.2388352595758526,0.467389332921739,6.35451159370473,2.82610202823881,16.08577990682851,-128013.078959091,simulation


k_eval,theta_eval
-0.425212606303152,0.630206510325516
-0.425177571276884,0.630206510325516
-0.425142536250617,0.630206510325516
-0.425107501224349,0.630206510325516
-0.425072466198082,0.630206510325516
-0.425037431171814,0.630206510325516


k_eval,theta_eval
-0.355352763925735,0.644662233111656
-0.355317728899468,0.644662233111656
-0.355282693873200,0.644662233111656
-0.355247658846933,0.644662233111656
-0.355212623820665,0.644662233111656
-0.355177588794398,0.644662233111656


In [53]:
test[log_like < cutoff_Sim2012]#[k_eval==min(k_eval)]
test[log_like < cutoff_Sim2012][k_eval==max(k_eval)]

k_eval,theta_eval,log_like
-0.415207603801901,0.641708542713568,725770.499080579


k_eval,theta_eval,log_like
-0.415207603801901,0.641708542713568,725770.499080579


In [ ]:
LogLikeSim2012 <- setDT(expand.grid(k_eval=seq(from=unique(LogLikeSim2012[log_like < cutoff_Sim2012][k_eval==min(k_eval), k_eval]), to=unique(LogLikeSim2012[log_like < cutoff_Sim2012][k_eval==max(k_eval), k_eval]), length.out = 2000), theta_eval=seq(from=unique(LogLikeSim2012[log_like < cutoff_Sim2012][theta_eval==min(theta_eval), theta_eval]), to=unique(LogLikeSim2012[log_like < cutoff_Sim2012][theta_eval==max(theta_eval), theta_eval]), length.out = 500)))
LogLikeSim2012[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_data2012$Distances, Assignments=biophys_par_data2012$Assignments, Sampled_reefs=biophys_par_data2012$Sampled_reefs, Reef_sizes=biophys_par_data2012$Reef_sizes, Adult_sample_proportions=biophys_par_data2012$Adult_sample_proportions), by=.(k_eval, theta_eval)]   #write profile results
fwrite(LogLikeSim2012, file="script_output/KernelFits/profiles/LikelihoodProfile-Biophysical-2012-NarrowSearch-08DayPLD.csv")

LogLikeSim2013 <- setDT(expand.grid(k_eval=seq(from=unique(LogLikeSim2013[log_like < cutoff_Sim2013][k_eval==min(k_eval), k_eval]), to=unique(LogLikeSim2013[log_like < cutoff_Sim2013][k_eval==max(k_eval), k_eval]), length.out = 2000), theta_eval=seq(from=unique(LogLikeSim2013[log_like < cutoff_Sim2013][theta_eval==min(theta_eval), theta_eval]), to=unique(LogLikeSim2013[log_like < cutoff_Sim2013][theta_eval==max(theta_eval), theta_eval]), length.out = 500)))
LogLikeSim2013[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_data2013$Distances, Assignments=biophys_par_data2013$Assignments, Sampled_reefs=biophys_par_data2013$Sampled_reefs, Reef_sizes=biophys_par_data2013$Reef_sizes, Adult_sample_proportions=biophys_par_data2013$Adult_sample_proportions), by=.(k_eval, theta_eval)]   #write profile results
fwrite(LogLikeSim2013, file="script_output/KernelFits/profiles/LikelihoodProfile-Biophysical-2013-NarrowSearch-08DayPLD.csv")

LogLikeSim2014 <- setDT(expand.grid(k_eval=seq(from=unique(LogLikeSim2014[log_like < cutoff_Sim2014][k_eval==min(k_eval), k_eval]), to=unique(LogLikeSim2014[log_like < cutoff_Sim2014][k_eval==max(k_eval), k_eval]), length.out = 2000), theta_eval=seq(from=unique(LogLikeSim2014[log_like < cutoff_Sim2014][theta_eval==min(theta_eval), theta_eval]), to=unique(LogLikeSim2014[log_like < cutoff_Sim2014][theta_eval==max(theta_eval), theta_eval]), length.out = 500)))
LogLikeSim2014[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_data2014$Distances, Assignments=biophys_par_data2014$Assignments, Sampled_reefs=biophys_par_data2014$Sampled_reefs, Reef_sizes=biophys_par_data2014$Reef_sizes, Adult_sample_proportions=biophys_par_data2014$Adult_sample_proportions), by=.(k_eval, theta_eval)]   #write profile results
fwrite(LogLikeSim2014, file="script_output/KernelFits/profiles/LikelihoodProfile-Biophysical-2014-NarrowSearch-08DayPLD.csv")

LogLikeSim2012_4 <- setDT(expand.grid(k_eval=seq(from=unique(LogLikeSim2012_4[log_like < cutoff_Sim2012_4][k_eval==min(k_eval), k_eval]), to=unique(LogLikeSim2012_4[log_like < cutoff_Sim2012_4][k_eval==max(k_eval), k_eval]), length.out = 2000), theta_eval=seq(from=unique(LogLikeSim2012_4[log_like < cutoff_Sim2012_4][theta_eval==min(theta_eval), theta_eval]), to=unique(LogLikeSim2012_4[log_like < cutoff_Sim2012_4][theta_eval==max(theta_eval), theta_eval]), length.out = 500)))
LogLikeSim2012_4[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_data2012_4$Distances, Assignments=biophys_par_data2012_4$Assignments, Sampled_reefs=biophys_par_data2012_4$Sampled_reefs, Reef_sizes=biophys_par_data2012_4$Reef_sizes, Adult_sample_proportions=biophys_par_data2012_4$Adult_sample_proportions), by=.(k_eval, theta_eval)]   #write profile results
fwrite(LogLikeSim2012_4, file="script_output/KernelFits/profiles/LikelihoodProfile-Biophysical-2012-4-NarrowSearch-08DayPLD.csv")

LogLikeSimNEM <- setDT(expand.grid(k_eval=seq(from=unique(LogLikeSimNEM[log_like < cutoff_SimNEM][k_eval==min(k_eval), k_eval]), to=unique(LogLikeSimNEM[log_like < cutoff_SimNEM][k_eval==max(k_eval), k_eval]), length.out = 2000), theta_eval=seq(from=unique(LogLikeSimNEM[log_like < cutoff_SimNEM][theta_eval==min(theta_eval), theta_eval]), to=unique(LogLikeSimNEM[log_like < cutoff_SimNEM][theta_eval==max(theta_eval), theta_eval]), length.out = 500)))
LogLikeSimNEM[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_dataNEM$Distances, Assignments=biophys_par_dataNEM$Assignments, Sampled_reefs=biophys_par_dataNEM$Sampled_reefs, Reef_sizes=biophys_par_dataNEM$Reef_sizes, Adult_sample_proportions=biophys_par_dataNEM$Adult_sample_proportions), by=.(k_eval, theta_eval)]   #write profile results
fwrite(LogLikeSimNEM, file="script_output/KernelFits/profiles/LikelihoodProfile-Biophysical-NEM-NarrowSearch-08DayPLD.csv")

LogLikeSimSWM <- setDT(expand.grid(k_eval=seq(from=unique(LogLikeSimSWM[log_like < cutoff_SimSWM][k_eval==min(k_eval), k_eval]), to=unique(LogLikeSimSWM[log_like < cutoff_SimSWM][k_eval==max(k_eval), k_eval]), length.out = 2000), theta_eval=seq(from=unique(LogLikeSimSWM[log_like < cutoff_SimSWM][theta_eval==min(theta_eval), theta_eval]), to=unique(LogLikeSimSWM[log_like < cutoff_SimSWM][theta_eval==max(theta_eval), theta_eval]), length.out = 500)))
LogLikeSimSWM[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_dataSWM$Distances, Assignments=biophys_par_dataSWM$Assignments, Sampled_reefs=biophys_par_dataSWM$Sampled_reefs, Reef_sizes=biophys_par_dataSWM$Reef_sizes, Adult_sample_proportions=biophys_par_dataSWM$Adult_sample_proportions), by=.(k_eval, theta_eval)]   #write profile results
fwrite(LogLikeSimSWM, file="script_output/KernelFits/profiles/LikelihoodProfile-Biophysical-SWM-NarrowSearch-08DayPLD.csv")

##genetics
#LogLikeGen2012 <- setDT(expand.grid(k_eval=seq(from=unique(LogLikeGen2012[log_like < cutoff_Gen2012][k_eval==min(k_eval), k_eval]), to=unique(LogLikeGen2012[log_like < cutoff_Gen2012][k_eval==max(k_eval), k_eval]), length.out = 2000), theta_eval=seq(from=unique(LogLikeGen2012[log_like < cutoff_Gen2012][theta_eval==min(theta_eval), theta_eval]), to=unique(LogLikeGen2012[log_like < cutoff_Gen2012][theta_eval==max(theta_eval), theta_eval]), length.out = 500)))
#LogLikeGen2012[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=par_data2012$Distances, Assignments=par_data2012$Assignments, Sampled_reefs=par_data2012$Sampled_reefs, Reef_sizes=par_data2012$Reef_sizes, Adult_sample_proportions=par_data2012$Adult_sample_proportions), by=.(k_eval, theta_eval)]   #write profile results
#fwrite(LogLikeGen2012, file="script_output/KernelFits/profiles/LikelihoodProfileGenetic2012NarrowSearch.csv")
#
#LogLikeGen2013 <- setDT(expand.grid(k_eval=seq(from=unique(LogLikeGen2013[log_like < cutoff_Gen2013][k_eval==min(k_eval), k_eval]), to=unique(LogLikeGen2013[log_like < cutoff_Gen2013][k_eval==max(k_eval), k_eval]), length.out = 2000), theta_eval=seq(from=unique(LogLikeGen2013[log_like < cutoff_Gen2013][theta_eval==min(theta_eval), theta_eval]), to=unique(LogLikeGen2013[log_like < cutoff_Gen2013][theta_eval==max(theta_eval), theta_eval]), length.out = 500)))
#LogLikeGen2013[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=par_data2013$Distances, Assignments=par_data2013$Assignments, Sampled_reefs=par_data2013$Sampled_reefs, Reef_sizes=par_data2013$Reef_sizes, Adult_sample_proportions=par_data2013$Adult_sample_proportions), by=.(k_eval, theta_eval)]   #write profile results
#fwrite(LogLikeGen2013, file="script_output/KernelFits/profiles/LikelihoodProfileGenetic2013NarrowSearch.csv")
#
#LogLikeGen2014 <- setDT(expand.grid(k_eval=seq(from=unique(LogLikeGen2014[log_like < cutoff_Gen2014][k_eval==min(k_eval), k_eval]), to=unique(LogLikeGen2014[log_like < cutoff_Gen2014][k_eval==max(k_eval), k_eval]), length.out = 2000), theta_eval=seq(from=unique(LogLikeGen2014[log_like < cutoff_Gen2014][theta_eval==min(theta_eval), theta_eval]), to=unique(LogLikeGen2014[log_like < cutoff_Gen2014][theta_eval==max(theta_eval), theta_eval]), length.out = 500)))
#LogLikeGen2014[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=par_data2014$Distances, Assignments=par_data2014$Assignments, Sampled_reefs=par_data2014$Sampled_reefs, Reef_sizes=par_data2014$Reef_sizes, Adult_sample_proportions=par_data2014$Adult_sample_proportions), by=.(k_eval, theta_eval)]   #write profile results
#fwrite(LogLikeGen2014, file="script_output/KernelFits/profiles/LikelihoodProfileGenetic2014NarrowSearch.csv")
#
#LogLikeGen2012_4 <- setDT(expand.grid(k_eval=seq(from=unique(LogLikeGen2012_4[log_like < cutoff_Gen2012_4][k_eval==min(k_eval), k_eval]), to=unique(LogLikeGen2012_4[log_like < cutoff_Gen2012_4][k_eval==max(k_eval), k_eval]), length.out = 2000), theta_eval=seq(from=unique(LogLikeGen2012_4[log_like < cutoff_Gen2012_4][theta_eval==min(theta_eval), theta_eval]), to=unique(LogLikeGen2012_4[log_like < cutoff_Gen2012_4][theta_eval==max(theta_eval), theta_eval]), length.out = 500)))
#LogLikeGen2012_4[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=par_data2012_4$Distances, Assignments=par_data2012_4$Assignments, Sampled_reefs=par_data2012_4$Sampled_reefs, Reef_sizes=par_data2012_4$Reef_sizes, Adult_sample_proportions=par_data2012_4$Adult_sample_proportions), by=.(k_eval, theta_eval)]   #write profile results
#fwrite(LogLikeGen2012_4, file="script_output/KernelFits/profiles/LikelihoodProfileGenetic2012_4NarrowSearch.csv")
#
#LogLikeGenNEM <- setDT(expand.grid(k_eval=seq(from=unique(LogLikeGenNEM[log_like < cutoff_GenNEM][k_eval==min(k_eval), k_eval]), to=unique(LogLikeGenNEM[log_like < cutoff_GenNEM][k_eval==max(k_eval), k_eval]), length.out = 2000), theta_eval=seq(from=unique(LogLikeGenNEM[log_like < cutoff_GenNEM][theta_eval==min(theta_eval), theta_eval]), to=unique(LogLikeGenNEM[log_like < cutoff_GenNEM][theta_eval==max(theta_eval), theta_eval]), length.out = 500)))
#LogLikeGenNEM[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=par_dataNEM$Distances, Assignments=par_dataNEM$Assignments, Sampled_reefs=par_dataNEM$Sampled_reefs, Reef_sizes=par_dataNEM$Reef_sizes, Adult_sample_proportions=par_dataNEM$Adult_sample_proportions), by=.(k_eval, theta_eval)]   #write profile results
#fwrite(LogLikeGenNEM, file="script_output/KernelFits/profiles/LikelihoodProfileGeneticNEMNarrowSearch.csv")
#
#LogLikeGenSWM <- setDT(expand.grid(k_eval=seq(from=unique(LogLikeGenSWM[log_like < cutoff_GenSWM][k_eval==min(k_eval), k_eval]), to=unique(LogLikeGenSWM[log_like < cutoff_GenSWM][k_eval==max(k_eval), k_eval]), length.out = 2000), theta_eval=seq(from=unique(LogLikeGenSWM[log_like < cutoff_GenSWM][theta_eval==min(theta_eval), theta_eval]), to=unique(LogLikeGenSWM[log_like < cutoff_GenSWM][theta_eval==max(theta_eval), theta_eval]), length.out = 500)))
#LogLikeGenSWM[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=par_dataSWM$Distances, Assignments=par_dataSWM$Assignments, Sampled_reefs=par_dataSWM$Sampled_reefs, Reef_sizes=par_dataSWM$Reef_sizes, Adult_sample_proportions=par_dataSWM$Adult_sample_proportions), by=.(k_eval, theta_eval)]   #write profile results
#fwrite(LogLikeGenSWM, file="script_output/KernelFits/profiles/LikelihoodProfileGeneticSWMNarrowSearch.csv")


In [70]:
par_data2014$Assignments==par_data2013$Assignments

Cabatoan,Caridad Cemetery,Caridad Proper,Elementary School,Haina,Hicgop South,Magbangon,Palanas,Poroc Rose,Poroc San Flower,San Agustin,Sitio Baybayon,Sitio Lonas,Tamakin Dacot,Visca,Wangag
TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,FALSE
TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
FALSE,TRUE,TRUE,TRUE,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
TRUE,TRUE,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,FALSE
TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
